In [243]:
import numpy as np

In [244]:
class Grid :
    def __init__(self,width,height,start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
    def setRAW(self,rewards, actions,walls):
        self.rewards = rewards
        self.actions = actions
        self.walls = walls
    def get_new_state(self,i,j,action):
        if action == "Up":
            nxtState = (i-1, j)
        elif action == "Down":
            nxtState = (i+1, j)
        elif action == "Left":
            nxtState = (i, j-1)
        else:
            nxtState = (i, j+1)
            # if next state legal
        if (nxtState[0] >= 0) and (nxtState[0] < self.height):
            if (nxtState[1] >= 0) and (nxtState[1] < self.width):
                if (not(nxtState[0],nxtState[1]) in self.walls):
                    return nxtState
        return (i,j)
    def value_function(self,policyMatrix=[0.25]*4,gamma=0.9):
        non_terminal_count = (self.width*self.height)-(len(self.rewards) + len(self.walls))
        coefficients = np.zeros((non_terminal_count,non_terminal_count))
        rhs = np.zeros((non_terminal_count))
        #prefix_matrix = prefix_sum_matrix()
        count = 0 
        prefix_matrix = [[0 for _ in range(self.width)] for _ in range(self.height)]
        for i in range(0,self.height):
            for j in range(0,self.width):
                if ((i,j) in self.walls or (i,j) in self.rewards):
                    prefix_matrix[i][j] = -1
                else:
                    prefix_matrix[i][j] = count
                    count += 1
        print("Prefix_matrix")
        print(prefix_matrix)
        
        count = 0
        for i in range(0,self.height):
            for j in range(0,self.width):
                if (not((i,j) in self.walls or (i,j) in self.rewards)):
                    #print("current state", i,j)
                    rewards_sum = 0
                    self_reference_coeff = 1
                    #print("current count",count)
                    for action_index in range(0,len(self.actions)):
                        pi_action = policyMatrix[action_index]
                        new_state = self.get_new_state(i,j,self.actions[action_index])
                        #print("new State",new_state)
                        prefix_output = prefix_matrix[new_state[0]][new_state[1]]
                        #print(prefix_output)
                        if (new_state in self.rewards):
                            rewards_sum += self.rewards.get(new_state) * pi_action
                            #print(rewards_sum)
                            #print(prefix_output)
                        elif (new_state[0] == i and new_state[1] == j):
                            self_reference_coeff -= (pi_action * gamma)
                            #print('self_reference_coff',self_reference_coeff)
                        else:
                            coefficient = pi_action*gamma*-1
                            #print('coefficient',coefficient)
                            coefficients[count][prefix_output] += coefficient 
                    coefficients[count][count] = self_reference_coeff
                    rhs[prefix_matrix[i][j]] = rewards_sum
                    count += 1
        print('Right Hand Side')
        print(rhs)
        print('Coefficients')
        print(coefficients)
        value_functions = np.linalg.solve(coefficients,rhs)
        return value_functions
    def action_function(self,value_function_output,policyMatrix=[0.25]*4,gamma=0.9):
        output_matrix = np.zeros((self.width*self.height,len(self.actions)))
        count = 0 
        prefix_matrix = [[0 for _ in range(self.width)] for _ in range(self.height)]
        for i in range(0,self.height):
            for j in range(0,self.width):
                if ((i,j) in self.walls or (i,j) in self.rewards):
                    prefix_matrix[i][j] = -1
                else:
                    prefix_matrix[i][j] = count
                    count += 1
        count = 0 
        for i in range(0,self.height):
            for j in range(0,self.width):
                if (prefix_matrix[i][j] == -1):
                        output_matrix[i*self.width+j][:] = 0
                else:
                    for action_index in range(0,len(self.actions)):
                        reward = 0
                        new_state = self.get_new_state(i,j,self.actions[action_index])
                        if (new_state in self.rewards):
                            reward += self.rewards.get(new_state)
                        value_function = 0
                        prefix_output = prefix_matrix[new_state[0]][new_state[1]]
                        if (prefix_output!= -1):
                            value_function = value_function_output[prefix_output]
                        output_matrix[i*self.width+j][action_index] = reward + gamma * value_function
        return output_matrix    
    def showValues(self):
        for i in range(0, BOARD_ROWS):
            print('----------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                out += str(self.state_values[(i, j)]) + ' | '
            print(out)
        print('----------------------------------') 
    def showBoard(self):
        #self.board[self.state] = 1
        for i in range(0, self.height):
            print('-----------------')
            out = '| '
            for j in range(0, self.width):
                if (i,j) in self.walls:
                    token = 'W'
                elif (i,j) in self.rewards:
                    token = str(self.rewards.get((i,j)))
                elif (i == self.i and j == self.j):
                    token = 'S'
                else:
                    token = 'N'
                token += ' ' + str((i,j)) + ' '
                out += token + ' | '
            print(out)
        print('-----------------')    

In [245]:
def standard_grid():
    g = Grid(4,3,(2,0))
    rewards = {(0,3):1,(1,3):-1}
    actions = ['Up','Down','Right','Left']
    walls = [(1,1)]
    g.setRAW(rewards,actions,walls)
    return g 

In [246]:
my_grid = standard_grid()
my_grid.showBoard()
v = my_grid.value_function()
print('State Value Function Values')
print(v)
q = my_grid.action_function(v)
print('Action Value Function Values')
print(q)

-----------------
| N (0, 0)  | N (0, 1)  | N (0, 2)  | 1 (0, 3)  | 
-----------------
| N (1, 0)  | W (1, 1)  | N (1, 2)  | -1 (1, 3)  | 
-----------------
| S (2, 0)  | N (2, 1)  | N (2, 2)  | N (2, 3)  | 
-----------------
Prefix_matrix
[[0, 1, 2, -1], [3, -1, 4, -1], [5, 6, 7, 8]]
Right Hand Side
[ 0.    0.    0.25  0.   -0.25  0.    0.    0.   -0.25]
Coefficients
[[ 0.55  -0.225  0.    -0.225  0.     0.     0.     0.     0.   ]
 [-0.225  0.55  -0.225  0.     0.     0.     0.     0.     0.   ]
 [ 0.    -0.225  0.775  0.    -0.225  0.     0.     0.     0.   ]
 [-0.225  0.     0.     0.55   0.    -0.225  0.     0.     0.   ]
 [ 0.     0.    -0.225  0.     0.775  0.     0.    -0.225  0.   ]
 [ 0.     0.     0.    -0.225  0.     0.55  -0.225  0.     0.   ]
 [ 0.     0.     0.     0.     0.    -0.225  0.55  -0.225  0.   ]
 [ 0.     0.     0.     0.    -0.225  0.    -0.225  0.775 -0.225]
 [ 0.     0.     0.     0.     0.     0.     0.    -0.225  0.55 ]]
State Value Function Values
[ 0.04